In [1]:
import pandas as pd
import re
import json

In [2]:
df = pd.read_csv('gigaword_train.csv')

In [3]:
CONTEXT = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
USER: Summarize in __WORDS__ words: __DOC__ 
ASSISTANT:"""

In [4]:
# Filters

print(len(df))
df = df[df['document'].apply(len)>50]
df = df[df['document'].apply(lambda x: 'UNK' not in x)]
df = df[df['summary'].apply(lambda x: 'UNK' not in x)]
df = df[df['summary'].apply(lambda x: len(x.split(' '))) < 15]
df = df[df['summary'].apply(lambda x: all([i.isalpha() for i in x.split(' ')]))]
print(len(df))

3803957
2089717


In [5]:
# Format
def train_format(row):
    _instruction = f"Summarize in {len(row[1].split(' '))} words"
    _input = row[0]
    _output = row[1]
    return {'instruction':_instruction,
           'input':_input,
           'output':_output}

In [6]:
df['train']=df.apply(train_format, axis=1)
df

,document,summary,train
0,australia 's current account deficit shrunk by...,australian current account deficit narrows sha...,"{'instruction': 'Summarize in 6 words', 'input..."
1,at least two people were killed in a suspected...,at least two dead in southern philippines blast,"{'instruction': 'Summarize in 8 words', 'input..."
3,south korea 's nuclear envoy kim sook urged no...,envoy urges north korea to restart nuclear dis...,"{'instruction': 'Summarize in 8 words', 'input..."
4,south korea on monday announced sweeping tax r...,skorea announces tax cuts to stimulate economy,"{'instruction': 'Summarize in 7 words', 'input..."
8,libyan leader moamer kadhafi monday promised w...,kadhafi promises wide political economic reforms,"{'instruction': 'Summarize in 6 words', 'input..."
...,...,...,...
3803949,chinese ambassador wang xuexian today urged is...,china urges return to peace process in middle ...,"{'instruction': 'Summarize in 9 words', 'input..."
3803951,south american prawns have been successfully b...,south american prawns bred in north china,"{'instruction': 'Summarize in 7 words', 'input..."
3803952,"the state duma , the lower house of the russia...",duma urges yeltsin to reconsider troop pullout...,"{'instruction': 'Summarize in 8 words', 'input..."
3803953,the u.s. justice department today rejected a t...,us justice department rejects special probe of,"{'instruction': 'Summarize in 7 words', 'input..."


In [7]:
with open('text-generation-webui/training/datasets/gigaword_vicuna.json', 'w') as fout:
    json.dump(list(df['train']), fout)